### IMPORTS

In [1]:
import pandas as pd
import numpy as np
import pandas_gbq
import datetime
from dateutil.relativedelta import relativedelta
import calendar
from gspread_pandas import Spread, conf
from Roster import *
import seaborn as sns

### CREDENCIALES

In [2]:
cred = conf.get_config('C:\\Users\\santiago.curat\\Pandas\\PEYA', 'PedidosYa-8b8c4d19f61c.json')

### CONSTANTES

In [11]:
# Carga
inicio = '2021-05-26'
fin = '2021-05-30'

### CARGA

#### QUERIES

In [12]:
q_carga = '''WITH sw_transaction AS (
    SELECT st.fullVisitorId AS full_id,
       st.visitId AS visit_id,
       st.platform_DH AS platform,
       DATE(st.date) AS day,
       st.shopId AS shop_id,
       st.transactions AS transactions
FROM `peya-bi-tools-pro.il_sessions.fact_sessions_events_transaction` AS st
WHERE DATE(st.date) BETWEEN DATE('{0}') AND DATE('{1}')
      AND st.cdCountry = 'Argentina'
      AND st.transactions > 0),
    sw_clicked AS (
    SELECT se.fullVisitorId AS full_id,
        se.visitId AS visit_id,
        se.platform_DH AS platform,
        DATE(se.date) AS day,
        se.cd20 AS shop_id,
        MAX(IF(REGEXP_CONTAINS(LOWER(REPLACE(se.cd110,' (AR)','ar ')),'descubrirar'),1,0)) AS desc_clicked,
        MAX(IF(REGEXP_CONTAINS(LOWER(REPLACE(se.cd110,' (AR)','ar ')),'exclusivosar'),1,0)) AS excl_clicked,
        MAX(CASE WHEN st.transactions IS NOT NULL AND st.transactions >= 1 AND REGEXP_CONTAINS(LOWER(REPLACE(se.cd110,' (AR)','ar ')),'descubrirar') THEN 1 ELSE 0 END) AS desc_transaction,
        MAX(CASE WHEN st.transactions IS NOT NULL AND st.transactions >= 1 AND REGEXP_CONTAINS(LOWER(REPLACE(se.cd110,' (AR)','ar ')),'exclusivosar') THEN 1 ELSE 0 END) AS excl_transaction
    FROM `peya-bi-tools-pro.il_sessions.fact_sessions_events` AS se
    LEFT JOIN sw_transaction AS st ON se.fullVisitorId = st.full_id
                                    AND se.visitId = st.visit_id
                                    AND se.platform_DH = st.platform
                                    AND DATE(se.date) = st.day
                                    AND se.cd20 = st.shop_id 
    WHERE DATE(se.date) BETWEEN DATE('{0}') AND DATE('{1}')
        AND REGEXP_CONTAINS(LOWER(se.eventAction),'swimlane.clicked')
        AND se.cdCountry = 'Argentina'
        AND (REGEXP_CONTAINS(LOWER(REPLACE(se.cd110,' (AR)','ar ')),'descubrirar') OR REGEXP_CONTAINS(LOWER(REPLACE(se.cd110,' (AR)','ar ')),'exclusivosar'))
    GROUP BY 1,2,3,4,5),
    sw_showed AS (
    SELECT se.fullVisitorId AS full_id,
        se.visitId AS visit_id,
        se.platform_DH AS platform,
        DATE(se.date) AS day,
        se.cdCountry AS country,
        se.cdCity AS city,
        se.cdArea AS area,
        MAX(IF(REGEXP_CONTAINS(LOWER(REPLACE(se.cd111,' (AR)','ar ')),'descubrirar') OR REGEXP_CONTAINS(LOWER(REPLACE(se.cd188,' (AR)','ar ')),'descubrirar'),1,0)) AS desc_shown,
        MAX(IF(REGEXP_CONTAINS(LOWER(REPLACE(se.cd111,' (AR)','ar ')),'exclusivosar') OR REGEXP_CONTAINS(LOWER(REPLACE(se.cd188,' (AR)','ar ')),'exclusivosar'),1,0)) AS excl_shown,
        SUM(IF(REGEXP_CONTAINS(LOWER(REPLACE(se.cd111,' (AR)','ar ')),'descubrirar') OR REGEXP_CONTAINS(LOWER(REPLACE(se.cd188,' (AR)','ar ')),'descubrirar'),1,0)) AS desc_shown_total,
        SUM(IF(REGEXP_CONTAINS(LOWER(REPLACE(se.cd111,' (AR)','ar ')),'exclusivosar') OR REGEXP_CONTAINS(LOWER(REPLACE(se.cd188,' (AR)','ar ')),'exclusivosar'),1,0)) AS excl_shown_total
    FROM `peya-bi-tools-pro.il_sessions.fact_sessions_events` AS se
    WHERE DATE(se.date) BETWEEN DATE('{0}') AND DATE('{1}')
        AND se.cdCountry = 'Argentina'
        AND se.eventAction IN ('shop_list.loaded','shop_list.updated')
        AND (REGEXP_CONTAINS(LOWER(REPLACE(se.cd111,' (AR)','ar ')),'descubrirar') OR REGEXP_CONTAINS(LOWER(REPLACE(se.cd111,' (AR)','ar ')),'exclusivosar'))
    GROUP BY 1,2,3,4,5,6,7)
SELECT ss.country AS Country,
       ss.city AS City,
       a.area_name AS Area,
       ss.platform AS Platform,
       SUBSTR(CAST(ss.day AS STRING),1,7) AS Month,
       ss.day AS Fecha,
       SUM(ss.excl_shown) AS Excl_Shown,
       SUM(ss.desc_shown) AS Desc_Shown,
       SUM(ss.excl_shown_total) AS Excl_Shown_Total,
       SUM(ss.desc_shown_total) AS Desc_Shown_Total,
       COUNT(DISTINCT(CASE WHEN sc.excl_clicked = 1 THEN CONCAT(IFNULL(sc.full_id,'-'),IFNULL(CAST(sc.visit_id AS STRING),'-'),IFNULL(sc.platform,'-'),IFNULL(CAST(sc.day AS STRING),'-')) ELSE NULL END)) AS Excl_Clicked,
       COUNT(DISTINCT(CASE WHEN sc.desc_clicked = 1 THEN CONCAT(IFNULL(sc.full_id,'-'),IFNULL(CAST(sc.visit_id AS STRING),'-'),IFNULL(sc.platform,'-'),IFNULL(CAST(sc.day AS STRING),'-')) ELSE NULL END)) AS Desc_Clicked,
       COUNT(DISTINCT(CASE WHEN sc.excl_transaction = 1 THEN CONCAT(IFNULL(sc.full_id,'-'),IFNULL(CAST(sc.visit_id AS STRING),'-'),IFNULL(sc.platform,'-'),IFNULL(CAST(sc.day AS STRING),'-')) ELSE NULL END)) AS Excl_Transaction,
       COUNT(DISTINCT(CASE WHEN sc.desc_transaction = 1 THEN CONCAT(IFNULL(sc.full_id,'-'),IFNULL(CAST(sc.visit_id AS STRING),'-'),IFNULL(sc.platform,'-'),IFNULL(CAST(sc.day AS STRING),'-')) ELSE NULL END)) AS Desc_Transaction,
       COALESCE(SUM(sc.excl_clicked),0) AS Excl_Clicked_Total,
       COALESCE(SUM(sc.desc_clicked),0) AS Desc_Clicked_Total,
       COALESCE(SUM(sc.excl_transaction),0) AS Excl_Transaction_Total,
       COALESCE(SUM(sc.desc_transaction),0) AS Desc_Transaction_Total
FROM sw_showed AS ss
LEFT JOIN sw_clicked AS sc ON ss.full_id = sc.full_id 
                           AND ss.visit_id = sc.visit_id
                           AND ss.platform = sc.platform
                           AND ss.day = sc.day
LEFT JOIN `peya-bi-tools-pro.il_core.dim_area` AS a ON ss.area = CAST(a.area_id AS STRING)
GROUP BY 1,2,3,4,5,6'''.format(inicio,fin)

In [13]:
# Descargo la data
hue_carga = pd.io.gbq.read_gbq(q_carga, project_id='peya-argentina', dialect='standard')

Downloading: 100%|███████████████████████████████████████████████████████████| 40983/40983 [00:08<00:00, 4922.01rows/s]


In [14]:
# Copio las bases
carga = hue_carga.copy()

#### CARGA DE TABLA PERSONAL

In [15]:
# Cargo la Tabla en la DDBB
pd.io.gbq.to_gbq(carga,destination_table='user_santiago_curat.swimlanes_argentina_resultados',project_id='peya-argentina',if_exists='append')

1it [00:18, 18.02s/it]


### ETAPA DE TRABAJO

#### QUERIES

In [26]:
q_res = '''SELECT slr.Country AS Country,
       slr.City AS City,
       slr.Area AS Area,
       slr.Platform AS Platform,
       slr.Month AS Month,
       DATE(slr.fecha) AS Fecha,
       slr.Excl_Shown AS Excl_Shown,
       slr.Desc_Shown AS Desc_Shown,
       slr.Excl_Shown_Total AS Excl_Shown_Total,
       slr.Desc_Shown_Total AS Desc_Shown_Total,
       slr.Excl_Clicked AS Excl_Clicked,
       slr.Desc_Clicked AS Desc_Clicked,
       slr.Excl_Transaction AS Excl_Transaction,
       slr.Desc_Transaction AS Desc_Transaction,
       slr.Excl_Clicked_Total AS Excl_Clicked_Total,
       slr.Desc_Clicked_Total AS Desc_Clicked_Total,
       slr.Excl_Transaction_Total AS Excl_Transaction_Total,
       slr.Desc_Transaction_Total AS Desc_Transaction_Total
FROM `peya-argentina.user_santiago_curat.swimlanes_argentina_resultados` AS slr'''

In [17]:
# Descargo la data
hue_res = pd.io.gbq.read_gbq(q_res, project_id='peya-argentina', dialect='standard')

Downloading: 100%|█████████████████████████████████████████████████████████| 289295/289295 [00:47<00:00, 6141.46rows/s]


In [18]:
# Copio las bases
res = hue_res.copy()

#### TRABAJO

In [20]:
# Doy formato a la fecha
res['Fecha'] = pd.to_datetime(res['Fecha'], format='%Y-%m-%d').dt.strftime('%Y-%m-%d')

In [23]:
# Creo columnas
val = ['Excl_Shown','Desc_Shown','Excl_Clicked','Desc_Clicked','Excl_Transaction','Desc_Transaction']
pt = res.pivot_table(index=['Fecha'],values=val,aggfunc='sum',fill_value=0).reset_index()
pt['CTR_Desc'] = round(pt['Desc_Clicked'] / pt['Desc_Shown'] * 100,2)
pt['CTR_Excl'] = round(pt['Excl_Clicked'] / pt['Excl_Shown'] * 100,2)
pt['CVR_Desc'] = round(pt['Desc_Transaction'] / pt['Desc_Clicked'] * 100,2)
pt['CVR_Excl'] = round(pt['Excl_Transaction'] / pt['Excl_Clicked'] * 100,2)
pt.replace([np.nan,np.inf,-np.inf],0,inplace=True)